In [275]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import os 
import cv2


In [276]:
#creating 2 numpy arrays x containing images and y containing labels  
def preprocess(foldername,e,h,l): 
 
 x=np.zeros((e+h+l,144,144,3)) 
 y=np.zeros((e+h+l,1,3))   

 samples=['Early_Blight','Healthy','Late_Blight']  

 for j in samples:  
    
    for i,pic in enumerate(os.listdir('/Users/shrutisekhar/Desktop/icg project/'+foldername+'/'+j)):  
        img=cv2.imread(os.path.join('/Users/shrutisekhar/Desktop/icg project/'+foldername+'/'+j,pic),cv2.COLOR_BGR2RGB) 
        img=cv2.resize(img,(144,144),interpolation=cv2.INTER_AREA)  
        if img is None: 
            print(f'error in {pic}')
            continue 
        img=img/255 #scaling all pixel between 0 and 1
        if j=='Early_Blight': 
             x[i]=img
        if j=='Healthy': 
             x[i+e]=img 
        if j=='Late_Blight': 
             x[i+e+h]=img   
        
        values=np.zeros((1,3))
        if j=='Early_Blight': 
             values[0,0]=1
             y[i]=values #0 for Early Blight  
        if j=='Healthy':  
             values[0,1]=1
             y[i+e]=values #1 for Healthy  
        if j=='Late_Blight':  
             values[0,2]=1
             y[i+e+h]=values #2 for Late Blight   
        y[i]=values
        
 return x,y

 


In [277]:
e=1303#no. of images in Early Blight
h=1303#no. of images in Healthy 
l=1132#no. of images in Late Blight 

x,y=preprocess('Training',e,h,l)
print(f'shape of x:{x.shape}') 
print(f'shape of y:{y.shape}')

shape of x:(3738, 144, 144, 3)
shape of y:(3738, 1, 3)


In [278]:
import tensorflow as tf 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout,Reshape,BatchNormalization
from tensorflow.keras.losses import CategoricalCrossentropy 
from tensorflow.keras.callbacks import EarlyStopping


In [279]:
model=Sequential([  
                   
                   Conv2D(32,(3,3),activation='relu',strides=(2,2),kernel_initializer='he_normal',input_shape=(144,144,3),name='conv1') ,  
                   MaxPooling2D(pool_size=(2,2),name='mp1'),  

                   Conv2D(64,(3,3),activation='relu',strides=(1,1),kernel_initializer='he_normal',name='conv2') ,
                   MaxPooling2D(pool_size=(2,2),name='mp2'),  
                   

                   Conv2D(128,(3,3),activation='relu',strides=(1,1),kernel_initializer='he_normal',name='conv3'),   
                   BatchNormalization(),
                   MaxPooling2D(pool_size=(2,2),name='mp3'), 

                   Flatten(name='flatten'), 
                   
                   Dense(128,activation='relu',name='dense1',kernel_initializer='he_normal') , 
                   Dropout(0.5,name='dropout4') , 
                   Dense(64,activation='relu',name='dense2',kernel_initializer='he_normal') ,
                   Dense(3,activation='softmax',name='output'),  

                   Reshape((1,3),name='reshape_output') 
                   
   ])  
model.compile(optimizer='rmsprop',loss=CategoricalCrossentropy,metrics=['accuracy']) 

 

/Users/shrutisekhar/Library/Python/3.9/lib/python/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [280]:
model.summary()

Model: "sequential_18"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1 (Conv2D)                  │ (None, 71, 71, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mp1 (MaxPooling2D)              │ (None, 35, 35, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2 (Conv2D)                  │ (None, 33, 33, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mp2 (MaxPooling2D)              │ (None, 16, 16, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv3 (Conv2D)                  │ (None, 14, 14, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_18          │ (None, 14, 14, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mp3 (MaxPooling2D)              │ (None, 7, 7, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 6272)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense1 (Dense)                  │ (None, 128)            │       802,944 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout4 (Dropout)              │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense2 (Dense)                  │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output (Dense)                  │ (None, 3)              │           195 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape_output (Reshape)        │ (None, 1, 3)           │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 905,155 (3.45 MB)

 Trainable params: 904,899 (3.45 MB)

 Non-trainable params: 256 (1.00 KB)

In [281]:
early_stopping=EarlyStopping(monitor='val_accuracy',patience=5,restore_best_weights=True) 


In [282]:
from sklearn.model_selection import train_test_split 

x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.2, random_state=42)

In [283]:
x_test,y_test=preprocess('Testing',162,102,141) 


In [284]:
history=model.fit(x_train,y_train,epochs=10,validation_data=(x_val,y_val))

Epoch 1/10
94/94 ━━━━━━━━━━━━━━━━━━━━ 5s 44ms/step - accuracy: 0.5574 - loss: 1.3317 - val_accuracy: 0.6150 - val_loss: 0.6481
Epoch 2/10
94/94 ━━━━━━━━━━━━━━━━━━━━ 4s 42ms/step - accuracy: 0.6141 - loss: 0.7010 - val_accuracy: 0.6003 - val_loss: 0.8436
Epoch 3/10
94/94 ━━━━━━━━━━━━━━━━━━━━ 4s 46ms/step - accuracy: 0.6360 - loss: 0.6619 - val_accuracy: 0.6337 - val_loss: 0.5942
Epoch 4/10
94/94 ━━━━━━━━━━━━━━━━━━━━ 4s 41ms/step - accuracy: 0.6670 - loss: 0.5929 - val_accuracy: 0.7166 - val_loss: 0.6095
Epoch 5/10
94/94 ━━━━━━━━━━━━━━━━━━━━ 4s 40ms/step - accuracy: 0.6999 - loss: 0.5418 - val_accuracy: 0.5561 - val_loss: 0.6613
Epoch 6/10
94/94 ━━━━━━━━━━━━━━━━━━━━ 4s 41ms/step - accuracy: 0.6954 - loss: 0.5289 - val_accuracy: 0.5307 - val_loss: 1.1255
Epoch 7/10
94/94 ━━━━━━━━━━━━━━━━━━━━ 4s 41ms/step - accuracy: 0.7091 - loss: 0.5178 - val_accuracy: 0.7380 - val_loss: 0.4637
Epoch 8/10
94/94 ━━━━━━━━━━━━━━━━━━━━ 4s 40ms/step - accuracy: 0.7203 - loss: 0.4545 - val_accuracy: 0.6123 - v

In [310]:
x_test,y_test=preprocess('Testing',162,102,141) 
y_pred_prob=model.predict(x_test)

y_pred=np.argmax(np.squeeze(y_pred_prob),axis=1) 
y_true=np.argmax(np.squeeze(y_test),axis=1)

13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step


In [311]:
print(y_pred_prob)

[[[0.4189158  0.03176513 0.549319  ]]

 [[0.16610582 0.15612084 0.6777733 ]]

 [[0.3767879  0.09513086 0.5280811 ]]

 ...

 [[0.19701816 0.05384817 0.7491336 ]]

 [[0.3377604  0.07053057 0.5917091 ]]

 [[0.3435043  0.05450713 0.6019886 ]]]


In [308]:
print(y_pred)

[2 2 2 2 2 2 0 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 0 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1 1 2 2 1 1 2 2 1 2 2 1 2 2 2 2 2 1 1 1 1
 1 1 2 2 1 1 1 2 1 2 1 1 1 1 1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 2 1 1 1 1
 1 1 2 1 1 1 1 1 1 1 2 2 2 1 2 1 1 1 1 1 1 2 1 1 1 2 2 1 1 2 1 2 1 1 2 1 2
 2 2 2 1 1 2 0 2 1 2 2 2 2 2 0 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 1 2 2 2 2 2 2 1 2 2 2 1 2 2 2 2 2 2 2 1 2 2 1 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 0
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]


In [312]:
print(y_true)

[2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]


In [313]:
 
print(np.shape(y_true) )
print(np.shape(y_pred))

(405,)
(405,)


In [315]:
#f1 score 
from sklearn.metrics import f1_score
f1 = f1_score(y_true, y_pred, average = 'weighted')
print(f"F1 Score: {f1}")

F1 Score: 0.8129531555028814
